[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_stl_decomposition/EMQA_stl_decomposition.ipynb)

# EMQA_stl_decomposition

STL decomposition of natural gas.

**Output:** `lecture1_decomposition.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from statsmodels.tsa.seasonal import STL

gas = fetch('NG=F')
monthly = gas.resample('ME').mean().dropna()
stl = STL(monthly, period=12, robust=True).fit()

fig, axes = plt.subplots(4, 1, figsize=(14, 11), sharex=True)
axes[0].plot(monthly.index, monthly, color=COLORS['green'], lw=1); axes[0].set_title('Observed', fontweight='bold'); axes[0].set_ylabel('USD')
axes[1].plot(stl.trend.index, stl.trend, color=COLORS['red'], lw=1.5); axes[1].set_title('Trend', fontweight='bold'); axes[1].set_ylabel('USD')
axes[2].plot(stl.seasonal.index, stl.seasonal, color=COLORS['blue'], lw=1); axes[2].set_title('Seasonal', fontweight='bold'); axes[2].set_ylabel('USD')
axes[2].axhline(0, color='gray', lw=0.5)
axes[3].plot(stl.resid.index, stl.resid, color=COLORS['gray'], lw=1); axes[3].set_title('Residual', fontweight='bold'); axes[3].set_ylabel('USD')
axes[3].axhline(0, color='gray', lw=0.5)

fig.tight_layout()
save_fig(fig, 'lecture1_decomposition.pdf')
plt.show()
